This script runs the built-in Notebooks for Semantic Model Check at the same time:
1. Best Practice Analyzer (BPA)
2. Memory Analyzer (equivalent to Vertipaq Analyzer)
3. Added Bonus: The Resultset of the BPA + Three DAX INFO View tables are saved into a Lakehouse
 
Process:
1. Create Lakehouse
2. Add Lakehouse to Notebook
3. Modify in the Python script the configuration for your Lakehouse Name, Workspace Name and Semantic Model Name
4. Run the Script
5. Check the result directly in the notebook or with the SQL Endpoint in your warehouse 

In [ ]:
import sempy.fabric as fabric
from datetime import datetime
import re
import pandas as pd

# ============================================================================
# CONFIGURATION
# ============================================================================
dataset = "Semantic Model Name"
workspace = "Workspace Name"
lakehouse = "Lakehouse_Name"

# Function to clean column names
def clean_column_name(col_name):
    col_name = str(col_name).replace('[', '').replace(']', '')
    col_name = col_name.replace(' ', '_')
    col_name = re.sub(r'[,;{}()\n\t=]', '', col_name)
    return col_name

# Function to save DataFrame with smart append/overwrite
def save_to_lakehouse(df, table_name, description="results"):
    """
    Saves DataFrame to lakehouse. Tries append first, creates table if needed.
    """
    full_table_name = f"{lakehouse}.{table_name}"
    
    # Convert pandas to Spark if needed
    if isinstance(df, pd.DataFrame):
        spark_df = spark.createDataFrame(df)
    else:
        spark_df = df
    
    print(f"Saving {description}...")
    
    try:
        # Try append first
        spark_df.write \
            .format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .saveAsTable(full_table_name)
        print(f"✓ Appended {len(df)} records")
    except:
        # Table doesn't exist, create it
        spark_df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(full_table_name)
        print(f"✓ Created table with {len(df)} records")

print("="*80)
print("STEP 1: Running Best Practice Analyzer...")
print("="*80)

# Run BPA analysis
try:
    bpa_results = fabric.run_model_bpa(
        dataset=dataset, 
        workspace=workspace,
        return_dataframe=True
    )
except TypeError:
    bpa_results = fabric.run_model_bpa(dataset=dataset, workspace=workspace)

if bpa_results is not None and len(bpa_results) > 0:
    # Add metadata
    bpa_results['analysis_timestamp'] = datetime.now()
    bpa_results['model_name'] = dataset
    bpa_results['workspace_name'] = workspace
    bpa_results.columns = [clean_column_name(col) for col in bpa_results.columns]

    # Save to lakehouse
    save_to_lakehouse(bpa_results, "bpa_analysis_results", "BPA results")

    print("\nSummary by Severity:")
    display(bpa_results.groupby('Severity').size())

print("\n" + "="*80)
print("STEP 2: Running Model Memory Analyzer...")
print("="*80)

memory_results = fabric.model_memory_analyzer(dataset=dataset, workspace=workspace)

print("\n" + "="*80)
print("STEP 3: Capturing Memory Statistics via DAX...")
print("="*80)

spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

try:
    # Table statistics
    print("\nGetting table statistics...")
    table_stats = fabric.evaluate_dax(dataset=dataset, workspace=workspace, dax_string="EVALUATE INFO.TABLES()")
    
    if table_stats is not None and len(table_stats) > 0:
        table_stats['analysis_timestamp'] = datetime.now()
        table_stats['model_name'] = dataset
        table_stats['workspace_name'] = workspace
        table_stats.columns = [clean_column_name(col) for col in table_stats.columns]
        
        # Handle old timestamps
        for col in ['ModifiedTime', 'RefreshedTime', 'StructureModifiedTime']:
            if col in table_stats.columns:
                table_stats[col] = pd.to_datetime(table_stats[col], errors='coerce')
                table_stats[col] = table_stats[col].where(table_stats[col] > pd.Timestamp('1900-01-01'), None)
        
        save_to_lakehouse(table_stats, "memory_analysis_tables", "table statistics")
        display(table_stats.head(10))
    
    # Column statistics
    print("\nGetting column statistics...")
    column_stats = fabric.evaluate_dax(dataset=dataset, workspace=workspace, dax_string="EVALUATE INFO.COLUMNS()")
    
    if column_stats is not None and len(column_stats) > 0:
        column_stats['analysis_timestamp'] = datetime.now()
        column_stats['model_name'] = dataset
        column_stats['workspace_name'] = workspace
        column_stats.columns = [clean_column_name(col) for col in column_stats.columns]
        
        # Handle old timestamps
        for col in ['ModifiedTime', 'RefreshedTime', 'StructureModifiedTime']:
            if col in column_stats.columns:
                column_stats[col] = pd.to_datetime(column_stats[col], errors='coerce')
                column_stats[col] = column_stats[col].where(column_stats[col] > pd.Timestamp('1900-01-01'), None)
        
        save_to_lakehouse(column_stats, "memory_analysis_columns", "column statistics")
        display(column_stats.head(10))
    
    # Measure statistics
    print("\nGetting measure statistics...")
    measure_stats = fabric.evaluate_dax(dataset=dataset, workspace=workspace, dax_string="EVALUATE INFO.MEASURES()")
    
    if measure_stats is not None and len(measure_stats) > 0:
        measure_stats['analysis_timestamp'] = datetime.now()
        measure_stats['model_name'] = dataset
        measure_stats['workspace_name'] = workspace
        measure_stats.columns = [clean_column_name(col) for col in measure_stats.columns]
        
        # Handle old timestamps
        for col in ['ModifiedTime', 'RefreshedTime', 'StructureModifiedTime']:
            if col in measure_stats.columns:
                measure_stats[col] = pd.to_datetime(measure_stats[col], errors='coerce')
                measure_stats[col] = measure_stats[col].where(measure_stats[col] > pd.Timestamp('1900-01-01'), None)
        
        save_to_lakehouse(measure_stats, "memory_analysis_measures", "measure statistics")
        display(measure_stats.head(10))
        
except Exception as e:
    print(f"Error: {str(e)}")

print("\n" + "="*80)
print("✓ ANALYSIS COMPLETE!")
print("="*80)
print("✓ All results saved to lakehouse and queryable via SQL Analytics Endpoint")